<a href="https://colab.research.google.com/github/ascle/colab_tcc2/blob/dev/3_limpeza-tratamento_dos_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color=green>2 - Transformando os dados</font>
***

**As transformações dos dados podem ser separadas em 4 grupos:**



*   Construtiva: adiciona ou copia dados;
*   Destrutiva: dados são substituídos ou deletados;
*   Estética: formatam a estrutura dos dados (datas, hora, endereço, etc…)
*   Estrutural: ocorre entre as colunas: renomeação, combinação e movimentação de colunas na tabela.






**Tipos de dados NaN**

Dados NaN são literalmente dados não preenchidos.


É muito importante, a partir do entendimento dos processos e regras de negócio, compreender a causa do surgimento destes dados NaN, pois esse entendimento é muito necessário para decidir como tratar esses dados da forma mais adequada possível.


Para incrementar nossa compreenssão sobre os dados NaN, formalmente podemos dividir os dados NaN em três grupos:

* Missing Completely at Random (MCAR): A probabilidade de uma instância estar ausente não depende nem de valores conhecidos e nem do próprio valor ausente. Isso entra no exemplo de valores ausentes na coluna RG por descuido do responsável pela coluna que simplesmente não registrou o dado e não pelo fato da pessoa não ter RG.

* Missing at Random (MAR): A probabilidade de uma instância estar ausente pode depender de valores conhecidos, mas não do próprio valor ausente. Por exemplo, esse caso entraria na situação do dado ser NaN na coluna Nome_do_Conjuge pelo fato da pessoa ter a informação ‘solteiro’ na coluna de Estado_Civil.

* Not Missing at Random (NMAR): A probabilidade de uma instância estar ausente pode depender do valor da própria variável. Podemos citar como exemplo o caso de uma coluna de notas de alunos de uma prova possuir dados NaN devido a alunos que zeraram a prova ou faltaram a prova final (também levando a uma nota 0).

**Referência:**

https://medium.com/data-hackers/tratamento-e-transforma%C3%A7%C3%A3o-de-dados-nan-uma-vis%C3%A3o-geral-e-pr%C3%A1tica-54efa9fc7a98

### Conectando no ambiente que será utilizado

Importação através o upload do arquivo

In [1]:
# Importação quando for feito o upload do arquivo

#import pandas as pd
#dados = pd.read_csv('dn.csv')
#dados.head()

Importação através do Google Drive

In [2]:
# Importando do Drive

from google.colab import drive
from os import chdir

drive.mount('/content/drive', force_remount=True)
chdir('/content/drive/MyDrive/UFS/tcc-2-ascle-ufs/bases/linkage_marco/')

Mounted at /content/drive


### Importando pandas e lendo o dataset do projeto

https://pandas.pydata.org/

In [3]:
import pandas as pd
import numpy as np
from random import randrange

dados = pd.read_csv('dn.csv')
dados.sort_index(axis=1, inplace=True)
dados.head()

,ANO,APGAR1,APGAR5,CODANOMAL,CONSPRENAT,CONSULTAS,ESCMAE,ESCMAE2010,ESCMAEAGR1,ESTCIVMAE,...,SERIESCMAE,SEXO,STCESPARTO,STDNNOVA,STTRABPART,TPAPRESENT,TPDOCRESP,TPFUNCRESP,TPMETESTIM,TPNASCASSI
0,2015,NaN,NaN,NaN,NaN,7_e_mais,Nenhuma,NaN,NaN,Casada,...,NaN,Feminino,NaN,Nao,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,NaN,NaN,NaN,NaN,de_4_a_6,4_a_7_anos,NaN,NaN,Casada,...,NaN,Feminino,NaN,Nao,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,NaN,NaN,NaN,NaN,7_e_mais,1_a_3_anos,NaN,NaN,Casada,...,NaN,Feminino,NaN,Nao,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,8.0,9.0,NaN,8.0,7_e_mais,Ignorado,Ignorado,Ignorado,Ignorado,...,NaN,Masculino,NaN,Sim,Nao,Ignorado,CPF,Medico,NaN,Outros
4,2015,2.0,4.0,NaN,5.0,de_4_a_6,Nenhuma,Sem_escolaridade,Sem_Escolaridade,Solteira,...,NaN,Masculino,NaN,Sim,Nao,Cefalico,COREN,Enfermeiro,Exame_fisico,Enfermeira/obstetriz


## <font color=green> 2.1 - Variáveis</font>
***


### 2) Anomalia detectada

##### Criando o campo <font color=blue>CODANOMAL_QTD</font> com a quantidade de anomalias

In [4]:
dados['CODANOMAL'].value_counts(dropna=False)

NaN                     197349
Q699                       258
Q668                       172
Q690                       150
Q549                       138
                         ...  
Q158Q423Q668Q700Q703         1
Q249Q668                     1
Q690Q703                     1
Q039Q054                     1
Q203                         1
Name: CODANOMAL, Length: 558, dtype: int64

In [5]:
dados['CODANOMAL_QTD'] = dados['CODANOMAL']
dados.drop(columns=['CODANOMAL'], inplace=True)
dados['CODANOMAL_QTD'] = dados['CODANOMAL_QTD'].apply(lambda x: x.count('Q') if (x != None and type(x) != float) else None)
#dados['CODANOMAL_QTD'].value_counts(dropna=False)

In [6]:
# Verificando os 0's
selecao = (dados['CODANOMAL_QTD'] == 0)
print(dados[selecao].CODANOMAL_QTD)
dados['CODANOMAL_QTD'] = dados['CODANOMAL_QTD'].apply(lambda x: 1 if (x == 0) else x)
#print(dados['CODANOMAL_QTD'].value_counts(dropna=False))

2733     0.0
6536     0.0
42671    0.0
Name: CODANOMAL_QTD, dtype: float64


In [7]:
dados['CODANOMAL_QTD'].fillna(0, inplace=True)
print(dados['CODANOMAL_QTD'].value_counts(dropna=False))

0.0    197349
1.0      2012
2.0       238
3.0        92
4.0        45
5.0        36
Name: CODANOMAL_QTD, dtype: int64


### 3 - Apgar no primeiro minuto

##### Removendo os registros com valores > 10

In [8]:
selecao = dados['APGAR1'] > 10
dados.drop(dados[selecao].index, inplace=True)

##### Tratando dados NaN

In [9]:
print('Quantidade NaN: {}'.format(dados['APGAR1'].isnull().sum()))
print('Percentual NaN: {}'.format(dados['APGAR1'].isnull().sum()/dados.shape[0]*100))
dados.dropna(subset=['APGAR1'], inplace=True, axis=0)

Quantidade NaN: 4630
Percentual NaN: 2.317665315112379


In [10]:
dados['APGAR1'].value_counts(dropna=False)

9.0     127207
8.0      39821
7.0       8673
10.0      6271
6.0       4638
5.0       3245
4.0       1997
3.0       1452
2.0        904
1.0        737
0.0        195
Name: APGAR1, dtype: int64

### 4 - Apgar no quinto minuto

##### Removendo os registros com valores > 10


In [11]:
selecao = dados['APGAR5'] > 10
dados.drop(dados[selecao].index, inplace=True)

##### Tratando NaN

In [12]:
print('Quantidade NaN: {}'.format(dados['APGAR5'].isnull().sum()))
print('Percentual NaN: {}'.format(dados['APGAR5'].isnull().sum()/dados.shape[0]*100))
dados.dropna(subset=['APGAR5'], inplace=True, axis=0)
dados['APGAR5'].value_counts(dropna=False)

Quantidade NaN: 81
Percentual NaN: 0.041508660448908476


10.0    126863
9.0      54265
8.0       9237
7.0       2475
6.0        775
5.0        443
1.0        421
4.0        211
3.0        140
2.0        132
0.0         97
Name: APGAR5, dtype: int64

### 5 - Escolaridade da mãe

##### Criação do campo <font color=blue>ESCMAEETL<font>

*Este novo campo irá unificar os dados dos demais campos referentes a escolaridade da mãe.*

Tratando SERIESCMAE

In [13]:
seriesmae = {
    0: 'Sem_Escolaridade',
    1: 'Fundamental_1',
    2: 'Fundamental_1',
    3: 'Fundamental_1',
    4: 'Fundamental_1',
    5: 'Fundamental_2',
    6: 'Fundamental_2',
    7: 'Fundamental_2',
    8: 'Fundamental_2',
}
dados['SERIESCMAE'] = dados['SERIESCMAE'].map(seriesmae)
print(dados['SERIESCMAE'].value_counts(dropna=False))

Fundamental_1       85288
Fundamental_2       58696
NaN                 51060
Sem_Escolaridade       15
Name: SERIESCMAE, dtype: int64


Tratando ESCMAE

In [14]:
escmae = {
    'Ignorado': 'Ignorado',
    '0': 'Ignorado',
    'Nenhuma': 'Sem_Escolaridade',
    '1_a_3_anos': 'Fundamental_1',
    '4_a_7_anos': 'Fundamental_2',
    '8_a_11_anos': 'Medio',
    '12_e_mais': 'Superior'
}
dados['ESCMAE'] = dados['ESCMAE'].map(escmae)
print(dados['ESCMAE'].value_counts(dropna=False))

Medio               105308
Fundamental_2        49835
Superior             29392
Fundamental_1         8626
Sem_Escolaridade      1168
NaN                    611
Ignorado               119
Name: ESCMAE, dtype: int64


Tratando ESCMAEAGR1

In [15]:
escmaeagr1 = {
    'Ignorado': 'Ignorado',
    'Sem_Escolaridade': 'Sem_Escolaridade',
    'Fundamental_I_Incompleto_ou_Inespecifico': 'Fundamental_1',
    'Fundamental_I_Incompleto': 'Fundamental_1',
    'Fundamental_I_Completo': 'Fundamental_1',
    'Fundamental_II_Incompleto_ou_Inespecifico': 'Fundamental_2',
    'Fundamental_II_Incompleto': 'Fundamental_2',
    'Fundamental_II_Completo': 'Fundamental_2',
    'Ensinomedio_Incompleto_ou_Inespecifico': 'Medio',
    'Ensino_Medio_Incompleto': 'Medio',
    'Ensino_Medio_Completo': 'Medio',
    'Superior_Incompleto': 'Superior',
    'Superior_Completo': 'Superior'
}
dados['ESCMAEAGR1'] = dados['ESCMAEAGR1'].map(escmaeagr1)
print(dados['ESCMAEAGR1'].value_counts(dropna=False))

Medio               85762
Fundamental_2       61892
Superior            29387
Fundamental_1       16095
Sem_Escolaridade     1168
NaN                   647
Ignorado              108
Name: ESCMAEAGR1, dtype: int64


Tratando ESCMAE2010

In [16]:
escmae2010 = {
    'Ignorado': 'Ignorado',
    'Sem_escolaridade': 'Sem_Escolaridade',
    'Fundamental_I_(1a_a_4a_serie)': 'Fundamental_1',
    'Fundamental_II_(5a_a_8a_serie)': 'Fundamental_2',
    'Ensinomedio_Incompleto_ou_Inespecifico': 'Medio',
    'Medio_(antigo_2o_Grau)': 'Medio',
    'Superior_incompleto': 'Superior',
    'Superior_completo': 'Superior'
}
dados['ESCMAE2010'] = dados['ESCMAE2010'].map(escmae2010)
print(dados['ESCMAE2010'].value_counts(dropna=False))

Medio               85762
Fundamental_2       61892
Superior            29387
Fundamental_1       16095
Sem_Escolaridade     1168
NaN                   647
Ignorado              108
Name: ESCMAE2010, dtype: int64


##### Criação do campo <font color=blue>ESCMAEETL<font>

In [17]:
dados['ESCMAEETL'] = np.nan
dados['ESCMAEETL'] = dados['ESCMAEAGR1']
dados['ESCMAEETL'].fillna(dados['ESCMAE2010'], inplace=True)
dados['ESCMAEETL'].fillna(dados['ESCMAE'], inplace=True)
dados['ESCMAEETL'].fillna(dados['SERIESCMAE'], inplace=True)
dados.drop(columns=['ESCMAEAGR1', 'ESCMAE2010', 'ESCMAE', 'SERIESCMAE'], inplace=True)
print(dados['ESCMAEETL'].value_counts(dropna=False))

Medio               85777
Fundamental_2       61895
Superior            29392
Fundamental_1       16097
Sem_Escolaridade     1168
NaN                   611
Ignorado              119
Name: ESCMAEETL, dtype: int64


##### Tratando Null e ignorado

In [18]:
dados['ESCMAEETL'].replace('Ignorado', None, inplace=True)
print('Quantidade NaN: {}'.format(dados['ESCMAEETL'].isnull().sum()))
print('Percentual NaN: {}'.format(dados['ESCMAEETL'].isnull().sum()/dados.shape[0]*100))
dados.dropna(subset=['ESCMAEETL'], inplace=True, axis=0)
print(dados['ESCMAEETL'].value_counts(dropna=False))

Quantidade NaN: 730
Percentual NaN: 0.3742457410322005
Medio               85777
Fundamental_2       61895
Superior            29392
Fundamental_1       16097
Sem_Escolaridade     1168
Name: ESCMAEETL, dtype: int64


##### Campo referente <font color=blue>ESCMAEETL_VALOR<font>

In [19]:
dados['ESCMAEETL_VALOR'] = dados['ESCMAEETL']
dados['ESCMAEETL_VALOR'].replace('Sem_Escolaridade', 1, inplace=True)
dados['ESCMAEETL_VALOR'].replace('Fundamental_1', 4, inplace=True)
dados['ESCMAEETL_VALOR'].replace('Fundamental_2', 8, inplace=True)
dados['ESCMAEETL_VALOR'].replace('Medio', 11, inplace=True)
dados['ESCMAEETL_VALOR'].replace('Superior', 15, inplace=True)

In [20]:
dados.ESCMAEETL_VALOR.isnull().sum()

0

In [21]:
# Removido devido a Tabela 4
# Pdf 30
# Na página 35 diz que é importante
dados.drop(columns=['ESCMAEETL'], inplace=True)

### 6 - Idade da mãe

Removido devido Tabela 1 (SILVA, 2022).

In [22]:
# pdf 27
dados.drop(columns=['IDADEMAE'], inplace = True)

###7) Idade do pai

In [23]:
# Removido devido Tabela 6  (SILVA, 2022)
# pdf 33
dados.drop(columns=['IDADEPAI'], inplace = True)

### 8- Local de nascimento

In [24]:
# Equivalente ao pandas.get_dummies
dados['LOCNASC'].replace('Ignorado', None, inplace=True)
dados.dropna(subset=['LOCNASC'], inplace=True)
dados['LOCNASC_DOMICILIO'] = dados['LOCNASC'].apply(lambda x: 1 if (x != None and str(x).__eq__('Domicilio')) else 0)
dados['LOCNASC_HOSPITAL'] = dados['LOCNASC'].apply(lambda x: 1 if (x != None and str(x).__eq__('Hospital')) else 0)
dados['LOCNASC_ESTAB_SAUDE'] = dados['LOCNASC'].apply(lambda x: 1 if (x != None and str(x).__eq__('Outro_Estab_Saude')) else 0)
dados['LOCNASC_OUTROS'] = dados['LOCNASC'].apply(lambda x: 1 if (x != None and str(x).__eq__('Outros')) else 0)
dados.drop(columns=['LOCNASC'], inplace=True)

### 9) Número de filhos vivos

In [25]:
# Removido devido Tabela 6  (SILVA, 2022)
print('Quantidade NaN: {}'.format(dados['QTDFILVIVO'].isnull().sum()))
print('Percentual NaN: {}'.format(dados['QTDFILVIVO'].isnull().sum()/dados.shape[0]*100))

dados.drop(columns=['QTDFILVIVO'], inplace=True)

Quantidade NaN: 21931
Percentual NaN: 11.285674588063358


###10) Número de consultas pré-natal

##### Merge e remoção da coluna <font color=blue>CONSULTAS</font>

In [26]:
dados['CONSPRENAT'].fillna(dados['CONSULTAS'], inplace=True)
dados['CONSPRENAT'].replace('Nenhuma', 0, inplace=True)
dados['CONSPRENAT'].replace('de_4_a_6', 5, inplace=True)

dados['CONSPRENAT'].replace('Ignorado', None, inplace=True)
dados['CONSPRENAT'].replace('7_e_mais', None, inplace=True)
dados.dropna(subset=['CONSPRENAT'], inplace=True)

dados.drop(columns=['CONSULTAS'], inplace=True)
print(dados['CONSPRENAT'].value_counts(dropna=False))


8.0     28024
7.0     27607
6.0     26639
10.0    21014
9.0     20131
5.0     19839
4.0     14277
3.0      9094
12.0     6149
11.0     5981
2.0      5261
1.0      2440
13.0     2090
14.0     1722
0.0      1601
15.0     1346
99.0      357
16.0      227
20.0      128
18.0      116
17.0      106
39.0       35
19.0       29
38.0       20
40.0       15
21.0        9
41.0        8
25.0        8
37.0        5
30.0        4
36.0        4
33.0        4
22.0        2
35.0        2
27.0        1
71.0        1
23.0        1
24.0        1
70.0        1
47.0        1
45.0        1
Name: CONSPRENAT, dtype: int64


### 11) Gestações anteriores

In [27]:
# Removido devido a Tabela 6 (SILVA, 2022)
# Pdf 33
dados.drop(columns=['QTDGESTANT'], inplace = True)

### 12) Número de partos cesáreos

### 13) Número de partos vaginais

In [28]:
# Removido devido a Tabela 6 (SILVA, 2022)
dados.drop(columns=['QTDPARTNOR'], inplace=True)

###<font color=red>14) Número de perdas fetais e abortos</font>

Os dados NaN equivalem a 20% do total de resgistros, tais dados foram substitudos por 0.

In [29]:
print('Quantidade NaN: {}'.format(dados['QTDFILMORT'].isnull().sum()))
print('Percentual NaN: {}'.format(dados['QTDFILMORT'].isnull().sum()/dados.shape[0]*100))
dados['QTDFILMORT'].fillna(0, inplace = True)
dados['QTDFILMORT'].value_counts(dropna=False)


Quantidade NaN: 38876
Percentual NaN: 20.008131713166684


0.0     156016
1.0      30088
2.0       6199
3.0       1401
4.0        358
5.0        126
6.0         62
7.0         19
8.0         15
9.0          7
11.0         3
14.0         3
10.0         2
20.0         1
12.0         1
Name: QTDFILMORT, dtype: int64

### <font color=blue>15 - Óbito Neo Natal </font>

####16 - Número de semanas de gestação

Existem dois campos relacionados ao número de semanas de gestação. Será feito o merge em SEMAGESTAC e o campo GESTACAO será excluído.

In [30]:
print('Quantidade NaN em GESTACAO: {}'.format(dados['GESTACAO'].isnull().sum()))
print('Percentual NaN em GESTACAO: {}'.format(dados['GESTACAO'].isnull().sum()/dados.shape[0]*100))

print('Quantidade NaN em SEMAGESTAC: {}'.format(dados['SEMAGESTAC'].isnull().sum()))
print('Percentual NaN em SEMAGESTAC: {}'.format(dados['SEMAGESTAC'].isnull().sum()/dados.shape[0]*100))

Quantidade NaN em GESTACAO: 1518
Percentual NaN em GESTACAO: 0.7812620624700851
Quantidade NaN em SEMAGESTAC: 1520
Percentual NaN em SEMAGESTAC: 0.7822913932506781


In [31]:
print(dados['GESTACAO'].value_counts(dropna=False))
print()
print(dados['SEMAGESTAC'].value_counts(dropna=False))

37_a_41_semanas        169770
32_a_36_semanas         16527
42_semanas_e_mais        4029
NaN                      1518
28_a_31_semanas          1480
22_a_27_semanas           796
Menos_de_22_semanas       180
Ignorado                    1
Name: GESTACAO, dtype: int64

39.0    66312
40.0    43732
38.0    30958
41.0    14593
37.0    14175
36.0     7098
35.0     4564
42.0     2627
34.0     2318
33.0     1596
NaN      1520
32.0      950
43.0      795
31.0      615
44.0      408
30.0      318
29.0      278
28.0      269
27.0      201
45.0      199
26.0      167
25.0      133
24.0      108
23.0       94
22.0       93
21.0       63
20.0       59
19.0       58
Name: SEMAGESTAC, dtype: int64


In [32]:
dados.drop(columns=['GESTACAO'], inplace=True)
dados.dropna(subset=['SEMAGESTAC'], inplace=True)

In [33]:
#dados.drop(columns=['QTDPARTCES'], inplace=True)

##### Convertendo Sim/Nao para 1/0

In [34]:
dados['OBT_NEONATAL'].replace('Sim', 1, inplace=True)
dados['OBT_NEONATAL'].replace('Nao', 0, inplace=True)

dados['OBT_NEONATAL'].value_counts(dropna=False)

0    191356
1      1425
Name: OBT_NEONATAL, dtype: int64

### 17- Peso ao nascer em gramas

### 18 - Sexo do recém-nascido

##### Setando None para os valores não informado

In [35]:
dados['SEXO'].replace('Ignorado,_nao_informado', None, inplace=True)
dados['SEXO_MASC'] = dados['SEXO'].apply(lambda x: 1 if (x != None and str(x).__eq__('Masculino')) else 0)
dados['SEXO_FEM'] = dados['SEXO'].apply(lambda x: 1 if (x != None and str(x).__eq__('Feminino')) else 0)
dados.dropna(subset=['SEXO'], inplace=True)
dados.drop(columns=['SEXO'], inplace=True)

### 19) Situação conjugal da mãe

In [36]:
#dados['ESTCIVMAE'].replace('Ignorado', None, inplace=True)
#dados['ESTCIVMAE_CASADA'] = dados['ESTCIVMAE'].apply(lambda x: 1 if (x != None and str(x).__eq__('Casada')) else 0)
#dados['ESTCIVMAE_SOLTEIRA'] = dados['ESTCIVMAE'].apply(lambda x: 1 if (x != None and str(x).__eq__('Solteira')) else 0)
#dados['ESTCIVMAE_UNIAO'] = dados['ESTCIVMAE'].apply(lambda x: 1 if (x != None and str(x).__eq__('Uniao_consensual_(versoes_anteriores)')) else 0)
#dados['ESTCIVMAE_DIVORCIADA'] = dados['ESTCIVMAE'].apply(lambda x: 1 if (x != None and str(x).__eq__('Separado_judicialmente/Divorciado')) else 0)
#dados['ESTCIVMAE_VIUVA'] = dados['ESTCIVMAE'].apply(lambda x: 1 if (x != None and str(x).__eq__('Viuva')) else 0)

dados['ESTCIVMAE_COMPANHEIRO'] = dados['ESTCIVMAE'].apply(lambda x: 1 if (x != None and (str(x).__eq__('Casada') or str(x).__eq__('Uniao_consensual_(versoes_anteriores)'))) else (None if (x == None) else 0))
#dados['ESTCIVMAE_COMPANHEIRO'] = dados['ESTCIVMAE'].apply(lambda x: None if (x == None) else x)
dados.drop(columns=['ESTCIVMAE'], inplace = True)

In [37]:
print('Quantidade NaN: {}'.format(dados['ESTCIVMAE_COMPANHEIRO'].isnull().sum()))
print('Percentual NaN: {}'.format((dados['ESTCIVMAE_COMPANHEIRO'].isnull().sum()/dados.shape[0])*100))

Quantidade NaN: 0
Percentual NaN: 0.0


In [38]:
dados.dropna(subset=['ESTCIVMAE_COMPANHEIRO'], inplace = True)

In [39]:
dados['ESTCIVMAE_COMPANHEIRO']

4         0
6         1
7         1
9         0
10        0
         ..
199767    1
199768    1
199769    0
199770    1
199771    0
Name: ESTCIVMAE_COMPANHEIRO, Length: 192741, dtype: int64

### 20) Tipo de apresentação do RN

In [40]:
dados['TPAPRESENT'].replace('Ignorado', None, inplace=True)
dados.dropna(subset=['TPAPRESENT'], inplace=True)
dados['TPAPRESENT_CEFALICO'] = dados['TPAPRESENT'].apply(lambda x: 1 if (x != None and str(x).__eq__('Cefalico')) else 0)
dados['TPAPRESENT_PELVICO'] = dados['TPAPRESENT'].apply(lambda x: 1 if (x != None and str(x).__eq__('Pelvica_ou_podalica')) else 0)
dados['TPAPRESENT_TRANSVERSA'] = dados['TPAPRESENT'].apply(lambda x: 1 if (x != None and str(x).__eq__('Transversa')) else 0)
dados.drop(columns=['TPAPRESENT'], inplace=True)

### 21) Tipo gravidez

In [41]:
dados['GRAVIDEZ'].replace('Ignorado', None, inplace=True)
dados.dropna(subset=['GRAVIDEZ'], inplace = True)
dados['GRAVIDEZ_VALOR'] = dados['GRAVIDEZ']
dados['GRAVIDEZ_VALOR'].replace('Unica', 1, inplace=True)
dados['GRAVIDEZ_VALOR'].replace('Dupla', 2, inplace=True)
dados['GRAVIDEZ_VALOR'].replace('Tripla_e_mais', 3, inplace=True)
dados.drop(columns=['GRAVIDEZ'], inplace=True)

### 22) Tipo de partos

In [42]:
dados.dropna(subset=['PARTO'], inplace=True)
dados['PARTO_CESAREO'] = dados['PARTO'].apply(lambda x: 1 if (x != None and str(x).__eq__('Cesareo')) else 0)
dados['PARTO_VAGINALL'] = dados['PARTO'].apply(lambda x: 1 if (x != None and str(x).__eq__('Vaginal')) else 0)
dados.drop(columns=['PARTO'], inplace=True)

### 23) Tipo de raça e cor da mãe

In [43]:
dados.drop(columns=['RACACORMAE'], inplace = True)
#dados['RACACORMAE_BRANCA'] = dados['RACACORMAE'].apply(lambda x: 1 if (x != None and x == 1) else 0)
#dados['RACACORMAE_PRETA'] = dados['RACACORMAE'].apply(lambda x: 1 if (x != None and x == 2) else 0)
#dados['RACACORMAE_AMARELA'] = dados['RACACORMAE'].apply(lambda x: 1 if (x != None and x == 3) else 0)
#dados['RACACORMAE_PARDA'] = dados['RACACORMAE'].apply(lambda x: 1 if (x != None and x == 4) else 0)
#dados['RACACORMAE_INDIGENA'] = dados['RACACORMAE'].apply(lambda x: 1 if (x != None and x == 5) else 0)

### Removendo coluna não utilizadas

In [44]:
dados.drop(columns=['IDADE_OBT', 'IDANOMAL', 'SEM_NOME', 'STCESPARTO', 'STDNNOVA',
                    'STTRABPART', 'TPDOCRESP', 'TPFUNCRESP', 'TPMETESTIM', 'TPNASCASSI'], inplace=True)

In [45]:
dados

,ANO,APGAR1,APGAR5,CONSPRENAT,OBT_NEONATAL,PESO,QTDFILMORT,QTDPARTCES,SEMAGESTAC,CODANOMAL_QTD,...,LOCNASC_OUTROS,SEXO_MASC,SEXO_FEM,ESTCIVMAE_COMPANHEIRO,TPAPRESENT_CEFALICO,TPAPRESENT_PELVICO,TPAPRESENT_TRANSVERSA,GRAVIDEZ_VALOR,PARTO_CESAREO,PARTO_VAGINALL
4,2015,2.0,4.0,5.0,0,820.0,1.0,NaN,26.0,0.0,...,0,1,0,0,1,0,0,1,1,0
6,2015,9.0,10.0,8.0,0,4200.0,1.0,NaN,39.0,0.0,...,0,1,0,1,1,0,0,1,1,0
7,2015,7.0,8.0,10.0,0,3700.0,0.0,0.0,40.0,0.0,...,0,0,1,1,1,0,0,1,0,1
9,2015,7.0,8.0,7.0,0,3550.0,1.0,0.0,37.0,0.0,...,0,1,0,0,1,0,0,1,0,1
10,2015,9.0,10.0,3.0,0,3800.0,0.0,NaN,39.0,0.0,...,0,0,1,0,1,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199767,2020,9.0,10.0,7.0,0,3020.0,0.0,0.0,41.0,1.0,...,0,0,1,1,1,0,0,1,1,0
199768,2020,9.0,9.0,6.0,0,2900.0,0.0,0.0,40.0,0.0,...,0,1,0,1,1,0,0,1,1,0
199769,2020,9.0,9.0,6.0,0,2975.0,0.0,0.0,41.0,0.0,...,1,0,1,0,1,0,0,1,1,0
199770,2020,9.0,9.0,12.0,0,3390.0,0.0,0.0,43.0,0.0,...,0,0,1,1,1,0,0,1,1,0


### Removendo dados duplicados

In [46]:
print("Registros: {}".format(dados.shape[0]))
print("Dados duplicados: {}".format(dados.duplicated().sum()))

Registros: 192484
Dados duplicados: 2464


In [47]:
dados.drop_duplicates(inplace=True, ignore_index=True)

In [48]:
print("Registros: {}".format(dados.shape[0]))
print("Dados duplicados: {}".format(dados.duplicated().sum()))

Registros: 190020
Dados duplicados: 0


### Salvando novo dataset

In [49]:
dados.shape

(190020, 24)

In [50]:
dados.isnull().sum()

ANO                          0
APGAR1                       0
APGAR5                       0
CONSPRENAT                   0
OBT_NEONATAL                 0
PESO                         0
QTDFILMORT                   0
QTDPARTCES               38712
SEMAGESTAC                   0
CODANOMAL_QTD                0
ESCMAEETL_VALOR              0
LOCNASC_DOMICILIO            0
LOCNASC_HOSPITAL             0
LOCNASC_ESTAB_SAUDE          0
LOCNASC_OUTROS               0
SEXO_MASC                    0
SEXO_FEM                     0
ESTCIVMAE_COMPANHEIRO        0
TPAPRESENT_CEFALICO          0
TPAPRESENT_PELVICO           0
TPAPRESENT_TRANSVERSA        0
GRAVIDEZ_VALOR               0
PARTO_CESAREO                0
PARTO_VAGINALL               0
dtype: int64

In [51]:
dados.to_csv("dn_etl.csv", sep=';', index=False)

### Referências

SILVA, T. S. L. de B. Fatores de risco ao nascer relacionados ao lóbito neonatal precoce e ao near miss neonatal: estudo de linkage entre o SINASC e o SIM. Dissertação (Dissertação de Doutorado) — Universidade Federal de Sergipe, 2022